# Accessing your data in StormDB using queries

Background/motivation here...

## Prerequisites

You must have the module `stormdb-python` in your Python path. A stable version is installed on the servers, but you may also want to clone a copy of the module into your project-folder.

In [ ]:
# These are optional, if you wish to modify your path
# import sys
# sys.path.insert(0, '/path/to/your/local/copy/of/stormdb-python)

## Import and initialise the Query-object

In [ ]:
from stormdb.access import Query

In [ ]:
# optional: see documentation for Query
Query?

In [ ]:
proj_name = 'MEG_EEG-Training'
qy = Query(proj_name)

To see what methods a Python-object offers, type the name of the instance, a dot, and hit Tab!

In [ ]:
qy.

## Do a search for series (file names) matching a particular pattern

In [ ]:
series = qy.filter_series()

## Exercise: Creating a "Neuromag-like" folder structure for raw files

Elekta Neuromag MEG data are saved on the acquisition computer as:

```bash
/neuro/data/sinuhe/neuromag_project_name/subj_ID/yymmdd/foo_raw.fif
/neuro/data/sinuhe/neuromag_project_name/subj_ID/yymmdd/bar_raw.fif
/neuro/data/sinuhe/neuromag_project_name/subj_ID/yymmdd/bar_raw-1.fif
/neuro/data/sinuhe/neuromag_project_name/subj_ID/yymmdd/bar_raw-2.fif
```

where the suffix `-1`, `-2`, ..., indicates that the acquisition `bar` was so long that it was split into a total of 3 files of maximum size 2 GB each.